# First Assignment (Chapter 2)

In [19]:
# imports
import numpy as np
from importlib import reload

import unyt

from oceanpy import constants
from oceanpy.forces import coriolis

from os_classes.ocea257 import chapter1
from os_classes import plotting as os_plotting


from bokeh import plotting
from bokeh import models

# Analytic

## 2-1

### Jupiter

Radius:  $R_J = C_J / 2 \pi$

In [8]:
circum_eq = 448600. * unyt.km # 
R_J = circum / 2 / np.pi  # Radius

### Omega

$\Omega = 2 \pi / t$

In [13]:
t_J = 9.9 * unyt.hr  # 1 day, i.e. full revolution
#
omega_J = 2 * np.pi / t_J
omega_J.to('1/s')

unyt_quantity(0.0001763, '1/s')

### Centrifugal acceleration

$a = \Omega^2 r$

In [12]:
a_centrifugal = omega_J**2 * R_J
a_centrifugal.to('m/s**2')

unyt_quantity(2.21903293, 'm/s**2')

### True acceleration

In [14]:
g_J_measured = 26.4 * unyt.m / unyt.s**2

In [15]:
g_J_true = g_J_measured + a_centrifugal
g_J_true

unyt_quantity(28.61903293, 'm/s**2')

----

## 2-2 Shinkasen  <font color='red'>(not assigned)</font> 

In [17]:
phi = 35 * unyt.deg

### Estimate the Coriolis acceleration 

#### Assume the train motion is purely horizontal

$\frac{dv}{dt} = -f u$

$f = 2 \Omega \sin\phi$

In [18]:
u_train = 185 * unyt.km / unyt.hr

In [22]:
f = coriolis.coriolis_parameter(phi)
f.to('1/s')

unyt_quantity(8.36518044e-05, '1/s')

In [26]:
# Acceleration
dvdt = -f * u_train
dvdt.to('m/s**2')

unyt_quantity(-0.00429877, 'm/s**2')

### This is $\approx 5 \times 10^{-4}$ of the gravitational acceleration

In [27]:
3*unyt.m * 5e-4

unyt_quantity(0.0015, 'm')

### For a track 3m wide, this implies a 1mm deflection.  Probably ignorable.

----

## 2-3 Cannonball

In [38]:
phi_London = (51. + 31./60)*unyt.deg
Dt = 25 * unyt.s
u_canonball = 120 * unyt.m / unyt.s

### Deflection $\Delta y$

#### Coriolis

$\frac{dv}{dt} = -f u$

$f = 2 \Omega \sin\phi$

In [32]:
f_London = coriolis.coriolis_parameter(phi_London)
f_London.to('1/s')

unyt_quantity(0.00011416, '1/s')

In [36]:
dvdt_London = -f_London * u_canonball
dvdt_London.to('m/s**2')

unyt_quantity(-0.01369967, 'm/s**2')

In [40]:
v_L_approx = dvdt_London * Dt
v_L_approx.to('m/s')

unyt_quantity(-0.34249173, 'm/s')

### Because $v \ll u$, we will ignore 2nd order effects during the $\Delta t = 25$s and take $dv/dt$ as constant.


#### Therefore, integrating and noting $v=0$ at $t=0$, we have

$\Delta y = \frac{1}{2} \frac{dv}{dt} t^2$

In [41]:
Dy = 0.5 * dvdt_London * Dt**2
Dy.to('m')

unyt_quantity(-4.28114668, 'm')

### The cannonball is deflected 4m to the right, i.e. S (assuming it is headed E)

### Other locations

In [43]:
def deflect(phi, Dt = 25 * unyt.s, u_canonball = 120 * unyt.m / unyt.s):
    # Coriolis parameter
    f = coriolis.coriolis_parameter(phi)
    # dv/dt
    dvdt = -f * u_canonball
    # Integrate
    Dy = 0.5 * dvdt * Dt**2
    # Return
    return Dy

### Murmansk

In [44]:
phi_murmansk = (68. + 52./60)*unyt.deg
Dy_murmansk = deflect(phi_murmansk)
Dy_murmansk

unyt_quantity(-5.10126301, 'm')

### Nairobi

In [45]:
phi_nairobi = -(1. + 18./60)*unyt.deg
Dy_nairobi = deflect(phi_nairobi)
Dy_nairobi

unyt_quantity(0.12407913, 'm')

### Any phi

In [50]:
phis = np.linspace(-90, 90, 1000)*unyt.deg

In [51]:
all_deflect = deflect(phis)

In [53]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Cannonball', x_axis_label='phi (deg)',
                    #x_axis_type="log", y_axis_type="log",
                   y_axis_label='Deflection (m)')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.line(phis, all_deflect, legend='Cannonball', line_width=2)
#p.scatter(kdx_vals, error_fb, legend='fb', line_width=2, color='red')
#p.scatter(kdx_vals, error_sc, legend='sc', line_width=2, color='green')
#p.scatter(kdx_vals, error_4, legend='4', line_width=2, color='black')


p.legend.location = "top_right"

os_plotting.bokeh_set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...